# Automatic Speech Recognition (ASR)

## Challenges in ASR

1. Background noise
2. Variability of the speaker (pitch, volume)
3. Same word, different speeds
4. Word Boundaries
5. Spoken language vs written language 

> Words are constant, but utterances aren’t. Spectrograms of similar words pronounced by the same speaker may be more alike than Spectrograms of the same word pronounced by different speakers. 

[http://www.seas.upenn.edu/~cis391/Lectures/speech-rec.pdf]

### Speech Recognition: Task Dimensions

* Speaker Dependent, Independent, Adaptive
            Speaker dependent: System trained for current speaker
            Speaker independent: No modificiation per speaker
            Speaker Adaptive: adapts an initial model to speaker
* Read vs. dictation vs. conversational
* Quiet Conditions vs. various noise conditions
* Known microphone vs. unknown microphone
* Perplexity level
            Low perplexity: Average expected branching factor of
        grammar < 10-20
            High perplexity: Average expected branching factor of
        grammar > 100 


## Pipeline for ASR

<img src="assets/asr-pipeline.png" />

So far, with MFCC we can extract features from speech and we can address challenges 1 and 2. This features can be turned on phonetic representation or phonemes using an acoustic model. Phonemes can be translated into words using a Lexical Decoding or Lexicon. However there are systems capable to translate the acoustic model into words. This is a **design choice** indeed, and it depends on the dimensionality of the problem. 


### The acoustic model and problems with time: same word, different speeds : intro to HMM's

The problem we address here, is the common fact that the words are usually pronounced using a diferent time length. For instance, besides the different pronunciations in the word 'Hello", it doesn't usually take the same time saying it. Further, some words might sound the same and only from the acoustic model, without providing any detail about the context, are really hard to distinguish like HEAR or HERE. 

Hidden Markov Models (HMM) are specially good to address the problem of variability in length, since they are really good to find patterns through time. The training set could consists of all labelled words, phonemes, sounds or groups of words, in order to determine the likelihood of a single word or phoneme. However this training becomes much more complicated when our dataset consists of utterances -phrases or whole sentences-. How could we this series of data be separated in training? Since a particular word may be connected to the previous and the next words, in order to train continuous utterances, HMM nodes are tied together as pairs which leads to an increase of dimensionality. When training words, the combinations are unbeareable, however training phonemes is a more accessible problem, since there are only 40 phonemes in English, only 1600 combinations are possible. Once the model is trained, it can be used to score new utterances. 

<img src="assets/sequence_possible_speech.png" /> 

### Adding knowledge: Language model

Which combinations of words are more reasonable? Even if from phonemes we can get words, we still can't solve language ambiguities in spelling and context, since we haven't taught to the model which combinations are more likely, or at least provide knowledge about the context, to allow the model to learn from itself. The language models just does this job: inject knowledge. 

Every single word can be thought as a probability of distribution over many different words. Each possible sequence can be calculated as the likelihood that a particular word could have been produced by the audio signal.

$$P(signal   |   w_1,w_2)$$

A statistical language model does precisely that. It provides a probability distribution over sequences of words. 

$$word_1, word_2, ... = argmax_{w_1 w_2 ...} {P(signal | w_1,w_2,...) * P(w_1,w_2,...)}$$

Even though, the dimensionality applying a statistical model is extremely huge, and some heuristics or approximate can be employed here: **It turns out that in practice, the words we speak at any time are primarily dependent upon the three to four previous words.**

### N-grams

N-grams are prob of single words "I", ordered pairs "I love" (bigrams), triples "I love Science", etc. With n-grams we can approximate the sequence probability using the chaing rule. 

$$ P("I", "love", "Science") = P("I") * P("love"|"I") * P("Science" | "I", "love") $$

Then we can score these probability along with the probabilities coming from the Acoustic model to remove language ambiguities from the sequence options and **provide a better estimate of the utterance give an text**. 

#### Quizz: computing bigrams

In the following series of quizes, you will work with 2-grams, or bigrams, as they are more commonly called. The objective is to create a function that calculates the probability that a particular sentence could occur in a corpus of text, based on the probabilities of its component bigrams. We'll do this in stages though:

* Quiz 1 - Extract tokens and bigrams from a sentence
* Quiz 2 - Calculate probabilities for bigrams
* Quiz 3 - Calculate the log probability of a given sentence based on a corpus of text using bigrams

##### Assumptions and terminology

* Utterance : 'I love language models'
* Tokens (word list from utterance + start tag + ending tag): 

$$ tokens = ["<s>", "I", "love", "language", "models", "</s>"]$$


* Bigrams: The bigrams for this sentence are represented as a list of lower case ordered pairs of tokens:


$$bigrams=[[("<s>", "i"), ("i", "love"), ("love", "language"), ("language", "models"), ("models", "</s>")]]$$


##### Quiz 1 Instructions

In the quiz below, write a function that returns a list of tokens and a list of bigrams for a given sentence. You will need to first break a sentence into words in a list, then add a token to the start and end of the list to represent the start and end of the sentence.

Your final lists should be in the format shown above and called out in the function doc string.

In [25]:
test_sentences = [
    'the old man spoke to me',
    'me to spoke man old the',
    'old man me old man me',
]


def get_token(sentence): 
    token_list = sentence.split(" ")
    token_list.insert(0, "<s>")
    token_list.append("</s>")   
    return token_list


def get_bigram(token_list): 
    
    pairs=[]  
    [pairs.append(token_list[i]) for i in range(1, len(token_list))]       
    return list(zip(token_list, pairs))

def sentence_to_bigrams(sentence):
    """
    Add start '<s>' and stop '</s>' tags to the sentence and tokenize it into a list
    of lower-case words (sentence_tokens) and bigrams (sentence_bigrams)
    :param sentence: string
    :return: list, list
        sentence_tokens: ordered list of words found in the sentence
        sentence_bigrams: a list of ordered two-word tuples found in the sentence
    """
    sentence_tokens = get_token(sentence)
    sentence_bigrams = get_bigram(sentence_tokens)
    
    return sentence_tokens, sentence_bigrams

[sentence_to_bigrams(sentence) for sentence in test_sentences] 

[(['<s>', 'the', 'old', 'man', 'spoke', 'to', 'me', '</s>'],
  [('<s>', 'the'),
   ('the', 'old'),
   ('old', 'man'),
   ('man', 'spoke'),
   ('spoke', 'to'),
   ('to', 'me'),
   ('me', '</s>')]),
 (['<s>', 'me', 'to', 'spoke', 'man', 'old', 'the', '</s>'],
  [('<s>', 'me'),
   ('me', 'to'),
   ('to', 'spoke'),
   ('spoke', 'man'),
   ('man', 'old'),
   ('old', 'the'),
   ('the', '</s>')]),
 (['<s>', 'old', 'man', 'me', 'old', 'man', 'me', '</s>'],
  [('<s>', 'old'),
   ('old', 'man'),
   ('man', 'me'),
   ('me', 'old'),
   ('old', 'man'),
   ('man', 'me'),
   ('me', '</s>')])]

##### Probabilities and Likelihoods with Bigrams

The probability of a series of words can be calculated from the chained probabilities of its history:

$$  P_{w_1, w_2, ...,w_n} = \prod_{i=1}^{n} P(w_i| w_1 w_2,...,w_{i-1})$$

The probabilities of sequence occurrences in a large textual corpus can be calculated this way and used as a **language model to add grammar and contextual knowledge to a speech recognition system**. However, there is a prohibitively large number of calculations for all the possible sequences of varying length in a large textual corpus.

To address this problem, we use the Markov Assumption to approximate a sequence probability with a shorter sequence.

###### Markov Assumption

In probability theory and statistics, the term Markov property refers to the memoryless property of a stochastic process. It is named after the Russian mathematician Andrey Markov.

**A stochastic process has the Markov property if the conditional probability distribution of future states of the process (conditional on both past and present states) depends only upon the present state, not on the sequence of events that preceded it**. A process with this property is called a Markov process. The term strong Markov property is similar to the Markov property, except that the meaning of "present" is defined in terms of a random variable known as a stopping time. Ex. Particles Brownian motion for times 0 ≤ t ≤ 2. Brownian motion has the Markov property, as the displacement of the particle does not depend on its past displacements.

The term Markov assumption is used to describe a model where the Markov property is assumed to hold, such as a hidden Markov model.

A Markov random field extends this property to two or more dimensions or to random variables defined for an interconnected network of items. An example of a model for such a field is the Ising model. A discrete-time stochastic process satisfying the Markov property is known as a Markov chain.

Thus, we could approximate

$$  P_{w_1, w_2, ...,w_n} \approx \prod_{i=1}^{n} P(w_i| w_{i-k}...w_{i-1})$$


We can calculate the probabilities by using counts of the bigrams (numerator) and individual tokens (denominator). The counts are represented below with the c() operator:

$$  P_{w_i|w_{i-1}} = \dfrac{c(w_{i-1}, w_i)}{c(w_{i-1})}$$

In the quiz below, write a function that returns a probability dictionary when given a lists of tokens and bigrams.

In [35]:
def bigrams_from_transcript(filename):
    """
    read a file of sentences, adding start '<s>' and stop '</s>' tags; Tokenize it into a list of lower case words
    and bigrams
    :param filename: string 
        filename: path to a text file consisting of lines of non-puncuated text; assume one sentence per line
    :return: list, list
        tokens: ordered list of words found in the file
        bigrams: a list of ordered two-word tuples found in the file
    """
    tokens = []
    bigrams = []
    with open(filename, 'r') as f:
        for line in f:
            line_tokens, line_bigrams = sentence_to_bigrams(line)
            tokens = tokens + line_tokens
            bigrams = bigrams + line_bigrams
    return tokens, bigrams


def sentence_to_bigrams(sentence):
    """
    Add start '<s>' and stop '</s>' tags to the sentence and tokenize it into a list
    of lower-case words (sentence_tokens) and bigrams (sentence_bigrams)
    :param sentence: string
    :return: list, list
        sentence_tokens: ordered list of words found in the sentence
        sentence_bigrams: a list of ordered two-word tuples found in the sentence
    """
    sentence_tokens = ['<s>'] + sentence.lower().split() + ['</s>']
    sentence_bigrams = []
    for i in range(len(sentence_tokens)-1):
        sentence_bigrams.append((sentence_tokens[i], sentence_tokens[i+1]))
    return sentence_tokens, sentence_bigrams

In [62]:
from collections import Counter

def sample_run():
    # sample usage by test code (this definition not actually run for the quiz)
    tokens, bigrams = bigrams_from_transcript('assets/transcript.txt')
    bg_dict = bigram_mle(tokens, bigrams)
    print(bg_dict)


def bigram_mle(tokens, bigrams):
    """
    provide a dictionary of probabilities for all bigrams in a corpus of text
    the calculation is based on maximum likelihood estimation and does not include
    any smoothing.  A tag '<unk>' has been added for unknown probabilities.
    :param tokens: list
        tokens: list of all tokens in the corpus
    :param bigrams: list
        bigrams: list of all two word tuples in the corpus
    :return: dict
        bg_mle_dict: a dictionary of bigrams:
            key: tuple of two bigram words, in order OR <unk> key
            value: float probability

    """
    
    bg_mle_dict = {}
    bg_mle_dict['<unk>'] = 0.
   
    # 530 words in the transcript   
    bigram_raw_counts = Counter(bigrams) # len = 515, Counter({('<s>', 'the'): 5, ('do', 'you'): 5...| c(wi−1,wi)
    token_raw_counts = Counter(tokens)   # len = 276, Counter({'the': 40, '</s>': 29, 'you': 17.... |  c(wi−1)    
    
    for bg in bigram_raw_counts: # for every possible bigram
        bg_mle_dict[bg] = bigram_raw_counts[bg] / token_raw_counts[bg[0]]
    return bg_mle_dict

In [63]:
sample_run()

{('escape', 'him'): 0.5, ('this', 'gentleman'): 0.16666666666666666, ('no', '</s>'): 0.5, ('therefore', 'entered'): 1.0, ('going', 'sir'): 1.0, ('towards', 'those'): 0.25, ('rave', 'sir'): 0.5, ('throat', 'swelled'): 1.0, ('honor', 'that'): 1.0, ('complete', 'this'): 1.0, ('as', 'the'): 0.125, ('do', 'yet'): 0.125, ('courage', 'to'): 1.0, ('asked', 'morrel'): 1.0, ('attracted', 'towards'): 1.0, ('say', 'that'): 1.0, ('<s>', 'noirtier'): 0.06896551724137931, ('kind', 'i'): 1.0, ('the', 'sight'): 0.025, ('go', 'do'): 0.5, ('swelled', 'his'): 1.0, ('near', 'the'): 1.0, ('which', 'became'): 0.2, ('remain', 'buried'): 1.0, ('<s>', 'at'): 0.034482758620689655, ('doctor', 'approached'): 0.5, ('we', 'may'): 0.5, ('davrigny', 'rushed'): 0.3333333333333333, ('his', 'pores'): 0.14285714285714285, ('unclosed', 'the'): 1.0, ('in', 'his'): 0.1111111111111111, ('a', 'powerful'): 0.14285714285714285, ('the', 'silent'): 0.025, ('into', 'my'): 1.0, ('noirtier', 'looked'): 0.25, ('to', 'you'): 0.07692307

##### Smoothing and logs

There are still a couple of problems to sort out before we use the bigram probability dictionary to calculate the probabilities of new sentences:


 1- **Some possible combinations may not exist in our probability dictionary but are still possible**. We don't want to multiply in a probability of 0 just because our original corpus was deficient. This is solved through "smoothing" 
-a technique used to smooth categorical data-. There are a number of methods for this, but a simple one is the Laplace smoothing with the "add-one" estimate where V is the size of the vocabulary for the corpus, i.e. the number of unique tokens:


$$  P_{add1(w_i|w_{i-1})} = \dfrac{c(w_{i-1}, w_i) + 1}{c(w_{i-1}) + V}$$


 2- Repeated multiplications of small probabilities can cause underflow problems in computers when the values become too small. To solve this, we will calculate all probabilities in log space:

$$ log(p_1 * p_2 * p_3 * p_4) = logp_1 + logp_2 + logp_3 + logp_4$$


##### Quiz 3 Instructions

Write a function that calculates the log probability for a given sentence, using this log probability dictionary. If all goes well, you should observe that more likely sentences yield higher values for the log probabilities.

In [64]:
from collections import Counter
import numpy as np

def bigrams_from_transcript(filename):
    """
    read a file of sentences, adding start '<s>' and stop '</s>' tags; Tokenize it into a list of lower case words
    and bigrams
    :param filename: string 
        filename: path to a text file consisting of lines of non-puncuated text; assume one sentence per line
    :return: list, list
        tokens: ordered list of words found in the file
        bigrams: a list of ordered two-word tuples found in the file
    """
    tokens = []
    bigrams = []
    with open(filename, 'r') as f:
        for line in f:
            line_tokens, line_bigrams = sentence_to_bigrams(line)
            tokens = tokens + line_tokens
            bigrams = bigrams + line_bigrams
    return tokens, bigrams


def sentence_to_bigrams(sentence):
    """
    Add start '<s>' and stop '</s>' tags to the sentence and tokenize it into a list
    of lower-case words (sentence_tokens) and bigrams (sentence_bigrams)
    :param sentence: string
    :return: list, list
        sentence_tokens: ordered list of words found in the sentence
        sentence_bigrams: a list of ordered two-word tuples found in the sentence
    """
    sentence_tokens = ['<s>'] + sentence.lower().split() + ['</s>']
    sentence_bigrams = []
    for i in range(len(sentence_tokens)-1):
        sentence_bigrams.append((sentence_tokens[i], sentence_tokens[i+1]))
    return sentence_tokens, sentence_bigrams

def bigram_add1_logs(transcript_file):
    """
    provide a smoothed log probability dictionary based on a transcript
    :param transcript_file: string
        transcript_file is the path filename containing unpunctuated text sentences
    :return: dict
        bg_add1_log_dict: dictionary of smoothed bigrams log probabilities including
        tags: <s>: start of sentence, </s>: end of sentence, <unk>: unknown placeholder probability
    """

    tokens, bigrams = bigrams_from_transcript(transcript_file)
    token_counts = Counter(tokens)
    bigram_counts = Counter(bigrams)
    vocab_count = len(token_counts)

    bg_addone_dict = {}
    for bg in bigram_counts:
        bg_addone_dict[bg] = np.log((bigram_counts[bg] + 1.) / (token_counts[bg[0]] + vocab_count)) # Laplace Smoothing
    bg_addone_dict['<unk>'] = np.log(1. / vocab_count)
    return bg_addone_dict

In [71]:
test_sentences = [
    'the old man spoke to me',
    'me to spoke man old the',
    'old man me old man me',
]

def sample_run():
    # sample usage by test code (this definition not actually run for the quiz)
    bigram_log_dict = bigram_add1_logs('assets/transcript.txt')
    for sentence in test_sentences:
        print('*** "{}"'.format(sentence))
        print(log_prob_of_sentence(sentence, bigram_log_dict))

def log_prob_of_sentence(sentence, bigram_log_dict):
    # get the sentence bigrams
    s_tokens, s_bigrams = sentence_to_bigrams(sentence)

    # add the log probabilites of the bigrams in the sentence
    total_log_prob = 0.
    
    for bg in s_bigrams:
        if bg in bigram_log_dict:
            total_log_prob = total_log_prob + bigram_log_dict[bg]
        else:
            total_log_prob = total_log_prob + bigram_log_dict['<unk>']
    return total_log_prob

In [72]:
sample_run()

*** "the old man spoke to me"
-34.8049553135
*** "me to spoke man old the"
-39.34280606
*** "old man me old man me"
-36.5989948127


## Unfolded view of ASR Architecture

<img src="assets/asr-arch.png" width="500"/>

[https://web.stanford.edu/class/cs224s/lectures/224s.17.lec3.pdf]

### Intro to Hidden Markov Models (HMM)

** Could we use frequency counts from the text to help compute the probability that the next letter in sequence would be a vowel? ** The HMM is a sequence model. A sequence model or sequence classifier is a model whose job is to assign a label or class to each unit in a sequence, thus mapping a sequence of observations to a sequence of labels. 

An HMM is a probabilistic sequence model: given a sequence of units (words, letters, morphemes, sentences,
whatever), they compute a probability distribution over possible sequences of labels and choose the best label sequence. Hidden Markov models (HMMs) are a way of relating a sequence of observations to a sequence of hidden classes or hidden states that explain the observations. 

In this part we present the mathematics of the HMM, beginning with the **Markov chain and then including the main three constituent algorithms: **

* the **Viterbi algorithm**. The process of discovering the sequence of hidden states, given the sequence of observations, is known as decoding or inference. The Viterbi algorithm is commonly used for decoding.

* the **Forward algorithm and the Baum-Welch or EM algorithm for unsupervised** (or semi-supervised) learning. The parameters of an HMM are the A transition probability matrix and the B observation likelihood matrix. Both can be trained with the Baum-Welch or forward-backward algorithm.

### Markov Chain = First-order observable Markov Model

First, let’s be more formal and view a Markov chain as a kind of probabilistic graphical model: a way of representing probabilistic assumptions in a graph. **A Markov chain is specified by the following components and embodies an important assumption about these probabilities: **

* A **set of N states** described by $Q$: $ Q = q_1, q_2…q_N$ the state at time $t$ is $q_t$


* A **transition probability matrix A**:
  * A set of probabilities $$A = a_{01}a_{02}…a_{n1}…a_{nn}$$
  * Each $a_{ij}$ represents the probability of transitioning from state i to state j
  * The	set	of	these	is	the	transition	probability	matrix	A 
  
  $$a_{ij} = P(q_t = j | q_{t−1} = i) 1≤ i, j ≤ N $$
  $$\sum_{j=1}^n a_{ij} = 1 ∀i$$
  
* Distinguished	start and end states. The start state may be a state but it can also be represented by a vector $\pi$ An initial distribution over probability of start states. If it so, the sum of all N $\pi$ start states = 1.   

<img src="assets/hmm.png" width="500" />

#### Markov's chain assumption

In a first-order Markov chain, **the probability of a particular state depends only on the previous state:**

  $$ P(q_i|q_1...q_{i−1}) = P(q_i|q_{i−1})$$
    
    
### Hidden Markov Model 

A Markov chain is useful when we need to compute a probability for a sequence of events that we can observe in the world. However, in many cases the events we are interested in, may not be directly observable in the world: 


To explain by example, I'll use an example from natural language processing. Imagine you want to know the probability of this sentence:

> I enjoy coffee

In a Markov model, you could estimate its probability by calculating:

$ P(WORD = I) * P(WORD = enjoy | PREVIOUS-WORD = I) * P(WORD = coffee| PREVIOUS-WORD = enjoy)$ 

Now, imagine we wanted to know the parts-of-speech tags of this sentence, that is, if a word is a past tense verb, a noun, etc. We did not observe any parts-of-speech tags in that sentence, but we assume they are there. Thus, we calculate what's the probability of the parts-of-speech tag sequence. In our case, the actual sequence is:

PRP-VBP-NN [https://cs.nyu.edu/grishman/jet/guide/PennPOS.html]

Since the parts-of-speech sequence are never directly observed, this can be considered as hidden states: we'd like to find the hidden sequence that best explains our observation. [https://stackoverflow.com/questions/10748426/what-is-the-difference-between-markov-chains-and-hidden-markov-model]

A HMM model is specified by the following components: 


* A **set of N states** described by $Q$: $ Q = q_1, q_2…q_N$ the state at time $t$ is $q_t$

* A **transition probability matrix A**:
  * A set of probabilities $$A = a_{01}a_{02}…a_{n1}…a_{nn}$$
  * Each $a_{ij}$ represents the probability of transitioning from state i to state j
  * The	set	of	these	is	the	transition	probability	matrix	A 
  
  $$a_{ij} = P(q_t = j | q_{t−1} = i) 1≤ i, j ≤ N $$
  $$\sum_{j=1}^n a_{ij} = 1 ∀i$$
  
* A sequence of $T$ observations $O = o_1 o_2 ...o_T$, each one drawn from a vocabulary $V=v_1, v_2,...,v_V$

* A sequence of observations likelihoods, also called emission probabilities, each expressing the probability of an observation being generated from a state i $B = b_i(o_t)$

* A special start state and end (final) state that are not associated with observations, together with transition probabilities $a_{01}a_{02} ...a_{0n}$ out of the start state and $a_{1F}a_{2F} ...a_{nF}$ into the end state.


##### HMM Assumptions

* **The probability of a particular state depends only on the previous state:**

  $$ P(q_i|q_1...q_{i−1}) = P(q_i|q_{i−1})$$
  

*  **Output Independence:** The probability of an output observation oi depends only on the state that produced the observation qi and not on any other states or any other observations. Thus, for Hidden Markov models, each hidden state produces only a single observation:


$$P(o_i|q_1 ...q_i,...,q_T , o_1,...,o_i,...,o_T ) = P(o_i|q_i)$$
    
#### Types of HMM's

Bakis HMMs are generally used to model temporal processes like speech.

<img src="assets/bakis-ergodic-hmm.png" width=500 />

### The Three Basic Problems we can address with HMMs

Now that we have seen the structure of an HMM, we turn to algorithms for computing things with them. An influential tutorial by [Rabiner (1989)](http://www.cs.ubc.ca/~murphyk/Bayes/rabiner.pdf), based on tutorials by Jack Ferguson in the 1960s, introduced the idea that **hidden Markov models should be characterized by three fundamental problems:**

* Problem 1 (Likelihood or evaluation): the	observation	sequence $O=(o_1 o_2 … o_T)$, and an HMM model $ \phi = (A,B)$, how do we efficiently compute $P(O|\phi)$, the probability of the observation sequence, given the model?



* Problem 2 (Decoding): Given	the	observation	sequence $O=(o_1 o_2 … o_T)$, and an HMM model $\phi = (A,B)$, how do we	choose	a	corresponding state sequence $Q=(q_1 q_2 … q_T)$ that	is	optimal	in	some sense?	-e.g. the seq of hidden states that better explains the observations-



* Problem 3 (Learning): How	do we adjust the model parameters $\phi =(A,B)$ to	maximize $P(O|\phi)$?